##**Exercise**


Instructor: Dr Mario Rosario Guarracino


---


**Load libraries**

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
# % cd "/content/drive/My Drive/2020 09 Cambridge course/PyNotebooks/"

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import sys
import scipy
import numpy as np
import pandas as pd
import sklearn as sk
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

# Classifiers
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

from sklearn.pipeline import Pipeline
from sklearn.feature_selection import RFE, RFECV

# Classifier metrics
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import confusion_matrix, classification_report, plot_confusion_matrix

# for plotting
import matplotlib.pyplot as plt
import seaborn as sns

**Read in the data**

This is a gene expression dataset derived from 224 blood samples of smokers and non-smokers which was used as a training set for the sbv improver systems toxicology computational challenge. More details on the data and challenge here:

- Belcastro, V., Poussin, C., Xiang, Y., Giordano, M., Tripathi, K.P., Boda, A., Balci, A.T., Bilgen, I., Dhanda, S.K., Duan, Z. and Gong, X., 2018. The sbv improver systems toxicology computational challenge: *Identification of human and species-independent blood response markers as predictors of smoking exposure and cessation status*. Computational Toxicology, 5, pp.38-51. https://www.sciencedirect.com/science/article/pii/S2468111317300348#s0220


In [ ]:
gene_exp = pd.read_csv('data/sc1_training_SvNCS.csv')

**Explore the data**

In [ ]:
gene_exp.head()

In [ ]:
print(gene_exp.shape)
gene_exp.describe
# This shows that the last two columns: gender and class are not numeric.

In [ ]:
gene_exp = gene_exp.drop('gender', axis=1) # drop gender

In [ ]:
X = gene_exp.iloc[:, :-1]
# class labels
y = gene_exp.iloc[:, -1]
print(y.dtype)
print(X.shape)
print(y.shape)

labels, counts = np.unique(y, return_counts=True)
print('labels', labels)
print('counts', counts)
print(y)
y = y.replace(labels, [0, 1]).array

print(y)


In [ ]:
print(gene_exp.isnull().sum().sum())
print(gene_exp.isna().sum().sum())

In [ ]:
# Divide the dataframe into attributes in X and disease diagnosis labels stored in y
X = gene_exp.iloc[:, :-1]
X.shape, y.shape

# scale X
scaler = StandardScaler()
# rows are patient samples and columns are genes so we will transpose it for 
X_scaled = scaler.fit_transform(X)

# PCA on scaled data
pca_sc = PCA(n_components=2)
X_pca_scaled = pca_sc.fit_transform(X_scaled)

plt.figure(figsize=(5, 5))
plt.scatter(X_pca_scaled[:, 0], X_pca_scaled[:, 1], c=y, cmap="Spectral", alpha=0.7)

# Percentage of variance explained for each components
print('Explained variance ratio (first two components): \n', '%s'
      % str(pca_sc.explained_variance_ratio_))

In [ ]:
# tSNE with 2 components
x_tsne = TSNE(n_components=2, random_state=1, perplexity=30).fit_transform(X_scaled)
x_tsne.shape

# store tsne output in a dataframe
df_for_tsne = pd.DataFrame({'tSNE1': x_tsne[:, 0], 'tSNE2': x_tsne[:, 1],
              'Diagnosis_CHD': y})

# seaborn for scatterplot
plt.figure(figsize=(6, 6))
sns.scatterplot(x="tSNE1", y="tSNE2",
			  hue='Diagnosis_CHD',
			  legend='full',
				palette =sns.color_palette("Set1", n_colors=2), s=45,
			  data=df_for_tsne)

### **RFE (Recursive feature elimination)**




In [ ]:
X = gene_exp.iloc[:, :-1].values

#Scaler
scaler = StandardScaler()
# classifier
model = SVC(kernel='linear', random_state=1)
# for RFE
estimator_for_rfe = SVC(kernel='linear', C=1, gamma= 0.01, random_state=1)
rfe = RFE(estimator_for_rfe, n_features_to_select=60, step=6000)

# RFECV for rfe with cross-validation 
# rfe = RFECV(estimator_for_rfe, min_features_to_select=60, step=6000, cv=2)

# crossvalidator
cv = StratifiedKFold(n_splits=10, random_state=1, shuffle=True)

In [ ]:
# baseline accuracy before RFE
model_run_bas = Pipeline([('scaler', scaler), ('SVM', model)])
scores_baseline = cross_val_score(model_run_bas, X, y, scoring='accuracy', cv=cv)
scores_baseline.mean()
print("Accuracy before rfe: ", scores_baseline.mean().round(2), u"\u00B1", scores_baseline.std().round(2))

In [ ]:
# RFE
model_run_rfe = Pipeline([('scaler', scaler), ('RFE', rfe), ('SVM', model)])
scores_rfe = cross_val_score(model_run_rfe, X, y, scoring='accuracy', cv=cv)

print("Accuracy Top60 rfe: ", scores_rfe.mean().round(2), u"\u00B1", scores_rfe.std().round(2))

**Load the Top 60 gene signature list identified in the sbv improver systems toxicology computational challenge** (Belcastro et al.)



In [ ]:
genes_sbvtop60 = pd.read_csv('data/sbvImprover_SvNCS_top60.csv')
X = gene_exp[genes_sbvtop60.columns].values

model_run_top_genes = Pipeline([('scaler', scaler), ('SVM', model)])
scores_best = cross_val_score(model_run_top_genes, X, y, scoring='accuracy', cv=cv)
print("Accuracy Top60 sbv: ", scores_best.mean().round(2), u"\u00B1", scores_best.std())

In [ ]:
# tSNE with 2 components and perplexity parameter set at 30
x_tsne = TSNE(n_components=2, random_state=1, perplexity=50).fit_transform(X)

# store tsne output in a dataframe
df_for_tsne = pd.DataFrame({'tSNE1': x_tsne[:, 0], 'tSNE2': x_tsne[:, 1],
              'Diagnosis_CHD': y})

# seaborn for scatterplot
plt.figure(figsize=(6, 6))
sns.scatterplot(x="tSNE1", y="tSNE2",
			  hue='Diagnosis_CHD',
			  legend='full',
				palette =sns.color_palette("Set1", n_colors=2), s=45,
			  data=df_for_tsne)



---

